# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bluff,-46.6000,168.3333,14.45,87,85,10.92,NZ,1675717152
1,1,calama,-22.4667,-68.9333,20.27,45,75,9.77,CL,1675717225
2,2,menongue,-14.6585,17.6910,17.41,83,99,1.43,AO,1675717424
3,3,buraydah,26.3260,43.9750,16.25,36,100,3.54,SA,1675717163
4,4,bambous virieux,-20.3428,57.7575,27.14,78,75,3.60,MU,1675717158


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
cities = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles ='OSM',
    frame_width = 700,
    frame_height = 500,
    size ='Humidity',
    scale = 0.7,
    color ='City'
)
# Display the map
cities

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_df = city_data_df.loc[(city_data_df['Max Temp'] > 20) & (city_data_df['Max Temp'] < 30) & (city_data_df['Cloudiness'] == 0) & (city_data_df['Wind Speed'] < 5.0), :]

# Drop any rows with null values
filtered_df = filtered_df.dropna()

# Display sample data
filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
49,49,lazaro cardenas,17.9583,-102.2000,28.04,58,0,4.89,MX,1675717322
72,72,saint-pierre,-21.3393,55.4781,25.82,78,0,3.09,RE,1675717451
100,100,tahoua,14.8888,5.2692,27.99,14,0,2.06,NE,1675717410
122,122,vila velha,-20.3297,-40.2925,27.19,80,0,3.09,BR,1675717409
126,126,abu dhabi,24.4667,54.3667,21.01,79,0,1.54,AE,1675717022
158,158,richards bay,-28.7830,32.0377,22.63,92,0,1.49,ZA,1675717496
166,166,kibala,9.1104,18.3463,23.97,46,0,0.51,TD,1675717498
170,170,yulara,-25.2406,130.9889,22.15,24,0,3.15,AU,1675717499
208,208,ascension,30.2169,-90.9001,23.38,45,0,4.12,US,1675717532
226,226,srivardhan,18.0333,73.0167,21.46,66,0,4.00,IN,1675717539


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
49,lazaro cardenas,MX,17.9583,-102.2000,58,
72,saint-pierre,RE,-21.3393,55.4781,78,
100,tahoua,NE,14.8888,5.2692,14,
122,vila velha,BR,-20.3297,-40.2925,80,
126,abu dhabi,AE,24.4667,54.3667,79,
158,richards bay,ZA,-28.7830,32.0377,92,
166,kibala,TD,9.1104,18.3463,46,
170,yulara,AU,-25.2406,130.9889,24,
208,ascension,US,30.2169,-90.9001,45,
226,srivardhan,IN,18.0333,73.0167,66,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [23]:
radius = 8000
limit = 20
longitude = -33.8698439
latitude = 151.2082848
filters = f"circle:{longitude},{latitude},{radius}"
bias = f"proximity:{longitude},{latitude}"
category = 'healthcare.pharmacy'
params = {
    "categories": category, 
    "apiKey": geoapify_key,   
    "filter":filters,
    "bias": bias,
    "limit": limit
}

base_url = "https://api.geoapify.com/v2/places"

place = requests.get(base_url, params=params)

place.json()

{'type': 'FeatureCollection', 'features': []}

In [12]:
radius = 10000
params = {
    "categories": 'accommodation.hotel', 
    "apiKey": geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df['Lat']
    longitude = hotel_df['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
lazaro cardenas - nearest hotel: No hotel found
saint-pierre - nearest hotel: No hotel found
tahoua - nearest hotel: No hotel found
vila velha - nearest hotel: No hotel found
abu dhabi - nearest hotel: No hotel found
richards bay - nearest hotel: No hotel found
kibala - nearest hotel: No hotel found
yulara - nearest hotel: No hotel found
ascension - nearest hotel: No hotel found
srivardhan - nearest hotel: No hotel found
salalah - nearest hotel: No hotel found
daund - nearest hotel: No hotel found
bodden town - nearest hotel: No hotel found
miraflores - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
49,lazaro cardenas,MX,17.9583,-102.2000,58,No hotel found
72,saint-pierre,RE,-21.3393,55.4781,78,No hotel found
100,tahoua,NE,14.8888,5.2692,14,No hotel found
122,vila velha,BR,-20.3297,-40.2925,80,No hotel found
126,abu dhabi,AE,24.4667,54.3667,79,No hotel found
158,richards bay,ZA,-28.7830,32.0377,92,No hotel found
166,kibala,TD,9.1104,18.3463,46,No hotel found
170,yulara,AU,-25.2406,130.9889,24,No hotel found
208,ascension,US,30.2169,-90.9001,45,No hotel found
226,srivardhan,IN,18.0333,73.0167,66,No hotel found


In [6]:
# Set parameters to search for a hotel
radius = # YOUR CODE HERE
params = {
    # YOUR CODE HERE
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = # YOUR CODE HERE
    params["bias"] = # YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = # YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kapaa - nearest hotel: Pono Kai Resort
hilo - nearest hotel: Dolphin Bay Hotel
banda - nearest hotel: #acnindiafy21
makakilo city - nearest hotel: Embassy Suites by Hilton Oahu Kapolei
kahului - nearest hotel: Maui Seaside Hotel
gat - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
tikaitnagar - nearest hotel: No hotel found
san quintin - nearest hotel: Jardines Hotel
santa rosalia - nearest hotel: Hotel del Real
narwar - nearest hotel: No hotel found
port hedland - nearest hotel: The Esplanade Hotel
roebourne - nearest hotel: No hotel found
saint-francois - nearest hotel: Chez Lily
capoterra - nearest hotel: Rosa Hotel
stolac - nearest hotel: Bregava
guerrero negro - nearest hotel: Plaza sal paraiso


,City,Country,Lat,Lng,Humidity,Hotel Name
45,kapaa,US,22.0752,-159.3190,84,Pono Kai Resort
51,hilo,US,19.7297,-155.0900,83,Dolphin Bay Hotel
63,banda,IN,25.4833,80.3333,52,#acnindiafy21
81,makakilo city,US,21.3469,-158.0858,81,Embassy Suites by Hilton Oahu Kapolei
152,kahului,US,20.8947,-156.4700,60,Maui Seaside Hotel
197,gat,IL,31.6100,34.7642,100,No hotel found
211,laguna,US,38.4210,-121.4238,79,Holiday Inn Express & Suites
240,tikaitnagar,IN,26.9500,81.5833,59,No hotel found
265,san quintin,MX,30.4833,-115.9500,74,Jardines Hotel
340,santa rosalia,MX,27.3167,-112.2833,56,Hotel del Real


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)